# Chapter 2: More on PyMC3

### Model context

In [2]:
import pymc3 as pm

with pm.Model() as model: # Variables defined in context are auto-assigned to a model
    parameter = pm.Exponential('poisson_param', 1, testval=0.5)
    data_generator = pm.Poisson('data_generator', parameter)

`parameter` -> Exponentially distributed r.v.

`data_generator` -> Poisson distributed r.v. with $\lambda$=`parameter`

In [3]:
with model: # To pick up a model again later
    data_plus_one = data_generator + 1 # Pois RV + 1

In [5]:
# Can examine variables outside of model context
parameter.tag.test_value # Initial value of variable
# ^ Can be specified at variable creation time

array(0.49999999904767284)

In [6]:
# Manipulating a separate model:
with pm.Model() as ab_testing:
    sigma = pm.Normal('std', 0, 1)
    p_A = pm.Uniform('p(A)', 0, 1)

Types of programming variables in PyMC3:
- Stochastic variables: variables drawn from a probability distribution (i.e. random variables).
- Deterministic variables: variables that are not random if the variables' parameters and components are known.

### Stochastic Variables

In [10]:
with pm.Model() as model:
    some_variable = pm.DiscreteUniform('discrete_uni_var', 0, 4)
    # Name attribute is used to retrieve posterior distribution later
    # Instead of initializing id. distr. variabels separately...
    # beta_1 = pm.Uniform('beta_1', 0, 1)
    # beta_2 = pm.Uniform('beta_2', 0, 1)
    # ... use the 'shape' argument!
    N = 10
    betas = pm.Uniform('betas', 0, 1, shape=N)


### Deterministic Variables

In [14]:
with model:
    deterministic_variables = pm.Deterministic('deterministic variables',
                                               some_variable*2)

In [15]:
with pm.Model() as model:
    lambda_1 = pm.Exponential('lambda_1', 1)
    lambda_2 = pm.Exponential('lambda_2', 1)
    tau = pm.DiscreteUniform('tau', lower=0, upper=10)

# Elementary operations implicitly create deterministic variables
new_deterministic_variable = lambda_1 + lambda_2
# if lambda_1 and lambda_2 are known, then we know new_det...

In [16]:
import numpy as np

n_data_points = 5
idx = np.arange(n_data_points)
with model:
    lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)
    
# lambda_ is deterministic
# if we know tau, lambda_1, lambda_2, we know lambda_

Stochastic variables behave like scalars/numpy arrays within deterministic variables -> can use Theano's tensor operations on them.

In [17]:
def multiply(x, y):
        return np.multiply(x, y) # Hadamard product

with model:
    stoch_1 = pm.Uniform('U_1', 0, 1)
    stoch_2 = pm.Uniform('U_2', 0, 1)

    det_1 = pm.Deterministic('Product', multiply(stoch_1, stoch_2))